# BERT Model Outcomes from 1980-1989

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Importing cleaned and processed data
df = pd.read_csv('../../data/clean_processed.csv')

In [ ]:
# Filtering data for only the 2000s
df = df[df['Year'].between(2000, 2009)]
df['Year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009])

In [ ]:
# Checking the demensions for filtered data 
df.shape

(972, 11)

In [ ]:
import re
from string import punctuation

# Light cleaning function to clean lyrics
def light_clean(text):
    if pd.isna(text):
        return ""
    # converting to lowercase
    text = text.lower()
    # removing special characters
    text = ''.join([char for char in text if char not in punctuation])
    # removing numbers 
    text = re.sub(pattern = r'\d+',
                  repl = '',
                   string = text)
    # removing extra whitespace
    text = re.sub(pattern = r'\s+',
                  repl = ' ',
                  string = text)
    
    cleaned = text  
    return cleaned

df['clean_lyrics'] = df['Lyrics'].apply(light_clean)

In [ ]:
# Creating a list of cleaned lyrics
lyrics = df['clean_lyrics'].tolist()

# Build and Fit BERTopic Model

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [18]:
# Creating Stopword List

stopwords_nltk = stopwords.words('english')
lyrics_stopwords = stopwords_nltk + [
    'la', 'lo', 'lola', 'du', 'oh', 'yeah', 'la la', 'lo lo', 'du du', 'got', 'gonna',
    'ooh', 'get', 'take', 'da', 'know', 'like', 'one', 'said', 'ron',
    'let', 'go', 'come', 'want', 'toke', 'say', 'day', 'good', 'see', 'make',
    'niggas', 'nigga', 'ya', 'choo', 'na', 'uh', 'yuh', 'hmm', 'wanna',
    'doo', 'dat', 'woo', 'nah', 'whoomp', 'yo', 'whoo', 'whatta', 'wa',
    'johnny', 'boaw', 'th', 'whoa', 'eh', 'ooo', 'um', 'dum', 'thy', 'aa',
    'the', 'on', 'and', 'ta', 'pa', 'oop', 'ha', 'bop', 'ah', 'yah', 'shes',
    'oo', 'im', 'ohoh', 'whats', 'huh', 'youre','dont', 'self', 'isnt', 'yeh', 
    'jo', 'cant', 'hes', 'uhuh', 'ive', 'yah', 'hmmmmm', 'ohh', 'woulda', 'nae',
    'lil', 'chh', 'ayy', 'aha', 'dit', 'ding', 'ling', 'feat', 'justin', 'mbabarara',
    'rainin', 'burnin', 'em', 'drake', 'sylvia', 'alejandro', 'mmm', 'watcha'
    'somethin', 'rumours', 'didnt', 'every', 'baby', 'girl', 'yes', 'wo',
    'bum', 'bum', 'ba', 'bedum', 'dye', 'dee', 'right', 'life', 'uhhuh',
    'cause', 'yuuuuuuu', 'aint', 'two', 'around']

vectorizer = CountVectorizer(stop_words=lyrics_stopwords)

In [9]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

In [19]:
# Dimensionality reduction
umap_model = UMAP(
    n_neighbors=10,       # local vs global structure
    n_components=10,      # dimensionality of reduced space
    min_dist=0.0,         # tighter clusters
    metric="cosine",
    random_state=42)

# Density-based clustering
hdbscan_model = HDBSCAN(
    min_cluster_size=20,  # how many docs per topic (granularity)
    min_samples=10,        # how strict the clustering is
    prediction_data=True,
    cluster_selection_method="leaf") # how clusters are calculated -> "leaf" results in more granular clusters 

# Topic model
topic_model_v3 = BERTopic(
    embedding_model= 'all-distilroberta-v1',  # can also try another model, e.g. all-mpnet-base-v2
    vectorizer_model=vectorizer,          # custom stopwords here
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    min_topic_size=20,                    # smallest amount of document allowed in a topic -> smaller number means more granular topics
    calculate_probabilities=True,
    verbose=True)

topics_v3, probabilities_v3 = topic_model_v3.fit_transform(lyrics)

2025-11-26 14:40:27,960 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

2025-11-26 14:49:34,434 - BERTopic - Embedding - Completed ✓
2025-11-26 14:49:34,436 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-26 14:49:37,931 - BERTopic - Dimensionality - Completed ✓
2025-11-26 14:49:37,933 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-26 14:49:38,010 - BERTopic - Cluster - Completed ✓
2025-11-26 14:49:38,019 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-26 14:49:38,324 - BERTopic - Representation - Completed ✓


In [20]:
topic_model_v3.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,550,-1_love_time_way_never,"[love, time, way, never, hey, man, back, could...",[ i aint no hater or nothing but he cant love ...
1,0,97,0_shit_hot_thoia_pop,"[shit, hot, thoia, pop, back, bay, club, give,...",[ snoop snoop when the pimps in the crib ma dr...
2,1,52,1_hope_light_sun_follow,"[hope, light, sun, follow, along, blue, need, ...",[ i hope you never lose your sense of wonder y...
3,2,49,2_love_never_time_bleeding,"[love, never, time, bleeding, keep, first, awa...",[ this time this place misused mistakes too lo...
4,3,43,3_watch_soulja_crank_boy,"[watch, soulja, crank, boy, ayyy, miya, shit, ...",[ miya hee miya ho miya hu miya haha miya hee ...
5,4,32,4_love_shake_name_kiss,"[love, shake, name, kiss, tonight, feel, ill, ...",[ my sexy love so sexy she makes the hairs on ...
6,5,31,5_cold_cry_sorry_goes,"[cold, cry, sorry, goes, used, wings, pretty, ...",[ you change your mind like a girl changes clo...
7,6,28,6_sugar_fly_step_mum,"[sugar, fly, step, mum, dem, jiggy, wit, man, ...",[ okay check it check it check it out its sant...
8,7,26,7_body_music_sir_way,"[body, music, sir, way, ride, move, ready, lov...",[part big boi ft sleepy brown the way you move...
9,8,22,8_without_lockdown_love_need,"[without, lockdown, love, need, never, keep, w...",[ yeah mmmm whoa oh yeah i dont wanna be be wi...


### Review Interpretable Topics

In [ ]:
# Songs about loneliness, self-doubt, hope for the future 
# embrace life, take chances, encouraging faith and hope 
lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[1]

# Topic Theme = Hope and Resilience 

[' i hope you never lose your sense of wonder you get your fill to eat but always keep that hunger may you never take one single breath for granted god forbid love ever leave you empty handed i hope you still feel small when you stand beside the ocean whenever one door closes i hope one more opens promise me that youll give faith a fighting chance and when you get the choice to sit it out or dance i hope you dance i hope you dance i hope you never fear those mountains in the distance never settle for the path of least resistance living might mean taking chances but theyre worth taking loving might be a mistake but its worth making dont let some hellbent heart leave you bitter when you come close to selling out reconsider give the heavens above more than just a passing glance and when you get the choice to sit it out or dance i hope you dance i hope you dance i hope you dance i hope you dance time is a wheel in constant motion always rolling us along tell me who wants to look back on th

In [ ]:
# Enduring nature of love; how love persists through distance or time 
# Declaration of enduring love and support; regret not expressing love sooner 
# Declaration of deep love and commitment after overcoming past struggles/ separation
lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[2]

# Topic Theme = Enduring Love

[' this time this place misused mistakes too long too late who was i to make you wait just one chance just one breath just in case theres just one left cause you know you know you know i love you i loved you all along and i miss you been far away for far too long i keep dreaming youll be with me and youll never go stop breathing if i dont see you anymore on my knees ill ask last chance for one last dance cause with you id withstand all of hell to hold your hand id give it all id give for us give anything but i wont give up cause you know you know you know i love you i loved you all along and i miss you been far away for far too long i keep dreaming youll be with me and youll never go stop breathing if i dont see you anymore so far away so far away been far away for far too long so far away so far away been far away for far too long but you know you know you know i wanted i wanted you to stay cause i needed i need to hear you say that i love you i love you i loved you all along and i fo

In [ ]:
# Frustration and Confusion in a relationship with emotionally inconsistent partner
# Pain of end of relationship due to fade in passion despite love being there still
# Previous trauma making it hard to trust in new relationship
lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[5]

# Topic Theme = Emotional Turbulence and Vulnerability

[' you change your mind like a girl changes clothes yeah you pms like a bitch i would know and you overthink always speak cryptically i should know that youre no good for me cause youre hot then youre cold youre yes then youre no youre in then youre out youre up then youre down youre wrong when its right its black and its white we fight we break up we kiss we make up you you dont really wanna stay no you but you dont really wanna go oh youre hot then youre cold youre yes then youre no youre in then youre out youre up then youre down we used to be just like twins so in sync the same energy nows a dead battery used to laugh used to laugh bout nothing bout nothing now youre plain now youre plain boring boring i should know i should know that youre not gonna change cause youre hot then youre cold youre yes then youre no youre in then youre out youre up then youre down youre wrong when its right its black and its white we fight we break up we kiss we make up you you dont really wanna stay n

In [ ]:
# Dancing and Atrraction
# Fun party lifestyle and attraction 
# Energy and connection in club setting 
lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[7]

# Topic Theme = Casual Romance w/ party life 

['part big boi ft sleepy brown the way you move ready for action nip it in the bud we never relaxing outkast is everlasting not clashing not at all but see my nigga went to do a little acting now thats for anyone asking give me one pass em drip drip drop there goes an eargasm now you cumming out the side of your face we tapping right into your memory banks thanks so click it or ticket lets see your seat belt fastened trunk rattling like two midgets in the backseat rassling speakerboxxx vibrate the tag make it sound like aluminum cans in a bag but i know yall wanted that can you feel that bass bass but i know yall wanted that can you feel that bass bass i like the way you move i like the way you move wooo i love the way you move i love the way i love the way i love the way you move i love the way you move wooo i love the way you move i love the way i love the way the whole room fell silent the girls all paused with glee turning left turning right are they looking at me well i was lookin

In [ ]:
# Couple deciding to stay together or break up 
# struggle of a failed romantic relationship
lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[8]


# Topic Theme = One Sided Love

[' yeah mmmm whoa oh yeah i dont wanna be be without you be without you i dont wanna live live without you live without you i dont wanna go go without you go without you i dont wanna be alone when we hooked up we sat down made an agreement we vowed that that wed always be together through whatever we said that no one would ever get between us this love would never ever leave us that was a while ago but now lately it feels like i mean i feel like all we do is fight every single night cant make it tight cant get it right i just wanna go back take it way back all the way back can we start again do it over can we straighten it out can we work it out cuz i dont wanna be be without you be without you i dont wanna live live without you live without you i dont wanna go go without you go without you i dont wanna be alone i dont wanna be be without you be without you i dont wanna live live without you live without you i dont wanna go go without you go without you i dont wanna be alone do you rea

In [ ]:
# Missing loved ones and feeling lost without them
# aftermath of a breakuo; holding on to past memories
lyrics_per_topic = topic_model_v3.get_representative_docs()
lyrics_per_topic[9]

# Topic Theme = Pain of Lost Love

[' theres another world inside of me that you may never see theres secrets in this life that i cant hide but somewhere in this darkness theres a light that i cant find well maybe its too far away yeah or maybe im just blind or maybe im just blind so hold me when im here right me when im wrong hold me when im scared and love me when im gone everything i am and everything in me wants to be the one you wanted me to be id never let you down even if i could id give up everything if only for your good so hold me when im here right me when im wrong you can hold me when im scared you wont always be there so love me when im gone love me when im gone when your education xray cannot see under my skin i wont tell you a damn thing that i could not tell my friends roaming through this darkness im alive but im alone and part of me is fighting this but part of me is gone so hold me when im here right me when im wrong hold me when im scared and love me when im gone everything i am and everything in me 

In [33]:
df['Topics'] = topics_v3
df.head()

,Year,Artist,Song Title,Rank,Lyrics,Nouns,Verbs,Adverbs,Corpus,Word Counts,Unique Word Counts,clean_lyrics,Topics
3923,2000,Faith Hill,Breathe,1,I can feel the magic floating in the air Bein...,magic air way sunlight face thought breeze arm...,feel float get watch dance sweep seem settle w...,never away just just away suddenly just just e...,feel magic float air get way watch sunlight da...,256,106,i can feel the magic floating in the air bein...,-1
3924,2000,Santana feat. Rob Thomas,Smooth,2,"Man, it's a hot one Like seven inches from th...",inch midday sun word muñequita reason reason s...,hear whisper melt stay say be give lift change...,so enough well so just so else out so enough w...,man hot like seven inch midday sun hear whispe...,351,128,man its a hot one like seven inches from the ...,-1
3925,2000,Joe,I Wanna Know,4,"Yeah, oh yeah Alright, oh, oh, oh It's amaz...",foot mmm time way lip breath time girl kind ma...,knock come get make feel kiss take know know t...,ever then away so so so so always never never ...,yeah oh yeah alright oh oh oh amazing knock fo...,478,144,yeah oh yeah alright oh oh oh its amazing how...,4
3926,2000,Vertical Horizon,Everything You Want,5,Somewhere there's speaking It's already comin...,back mind knee skid mark place angel thing tim...,be speak come rise get feed know use learn how...,somewhere already never now here exactly toget...,speak come oh rise mind feed know skinned knee...,334,119,somewhere theres speaking its already coming ...,-1
3927,2000,Destiny's Child,Say My Name,6,"Darkchild '99 Say my name, say my name If n...",name name one baby game name name baby change ...,say say say love say say be say say say love s...,there first around so back usually just there ...,darkchild baby love be runnin game actin kinda...,889,214,darkchild say my name say my name if no one i...,4
